In [1]:
import src.decorrelate as cluster_model
MODEL_NAME = 'EleutherAI/pythia-70m'
DATASET_NAME = 'NeelNanda/pile-10k'

N_DIMS = 512
SEED = 69_420
 
DEBUG = False
 
if DEBUG:
    N_DATASIZE = 300
    N_BLOCKS = 12
    STRING_SIZE_CUTOFF = 200
else:
    # It gets killed aroun 1_800 idk why. Maybe we have a problem with token truncation somewhere
    N_DATASIZE = 2_001 # TODO: was 3_000 for non-digraph
# 
    # N_CLUSTERS_MIN = int(0.5 * N_DIMS)
    # N_CLUSTERS_MAX = 10 * N_DIMS
    # TODO: DEL ME
    N_BLOCKS = 6
    STRING_SIZE_CUTOFF = 1_000

params = cluster_model.InterpParams(
	lattice_params=cluster_model.LatticeParams(
		top_layer_idx = -1,
        max_n_parents = 4
	),
    # quantization='4bit',
	seed=SEED,
    n_datasize=N_DATASIZE,
    n_blocks=N_BLOCKS,
    model_name=MODEL_NAME,
	model_n_dims=N_DIMS,
    dataset_name=DATASET_NAME,
    string_size_cutoff=STRING_SIZE_CUTOFF,
    quantization='4bit'
)

decomp = cluster_model.Decomposer(params)

/home/lev/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating decomposer with parameter data hash cache/data-1c7597d69d97d2128ba3828c410fd633cf9841c1/start.pkl
Creating decomposer with parameter lattice hash cache/correlation-a416ec458fa5589b619bfa2849adfc7b746abab8/start.pkl
Created dataset


In [2]:
# TODO: make this a bit nicer with hooks
# But we are changing to GPTNeoxLayer to
"""
   class GPTNeoXLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.use_parallel_residual = config.use_parallel_residual
        self.input_layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.post_attention_layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.post_attention_dropout = nn.Dropout(config.hidden_dropout)
        self.post_mlp_dropout = nn.Dropout(config.hidden_dropout)
        self.attention = GPTNeoXAttention(config)
        self.mlp = GPTNeoXMLP(config)

    def forward(
        self,
        hidden_states: Optional[torch.FloatTensor],
        attention_mask: Optional[torch.FloatTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = False,
        layer_past: Optional[Tuple[torch.Tensor]] = None,
        output_attentions: Optional[bool] = False,
    ):
        attention_layer_outputs = self.attention(
            self.input_layernorm(hidden_states),
            attention_mask=attention_mask,
            position_ids=position_ids,
            layer_past=layer_past,
            head_mask=head_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
        )
        attn_output = attention_layer_outputs[0]  # output_attn: attn_output, present, (attn_weights)
        attn_output = self.post_attention_dropout(attn_output)
        outputs = list(attention_layer_outputs[1:])

        if self.use_parallel_residual:
            # pseudocode:
            # x = x + attn(ln1(x)) + mlp(ln2(x))
            mlp_output = self.mlp(self.post_attention_layernorm(hidden_states))
            mlp_output = self.post_mlp_dropout(mlp_output)
            hidden_states = mlp_output + attn_output + hidden_states
            # Override the present state to be the pre-residual stream addition
            outputs[0] = mlp_output + attn_output
        else:
            raise NotImplementedError
            # pseudocode:
            # x = x + attn(ln1(x))
            # x = x + mlp(ln2(x))
            attn_output = attn_output + hidden_states
            mlp_output = self.mlp(self.post_attention_layernorm(attn_output))
            mlp_output = self.post_mlp_dropout(mlp_output)
            hidden_states = mlp_output + attn_output

        outputs = tuple(outputs)
        if use_cache:
            outputs = (hidden_states,) + outputs  # hidden_states, present, (attn_weights)
        else:
            outputs = (hidden_states,) + outputs[1:]  # hidden_states, (attn_weights)

        return outputs

"""

'\n   class GPTNeoXLayer(nn.Module):\n    def __init__(self, config):\n        super().__init__()\n        self.use_parallel_residual = config.use_parallel_residual\n        self.input_layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)\n        self.post_attention_layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)\n        self.post_attention_dropout = nn.Dropout(config.hidden_dropout)\n        self.post_mlp_dropout = nn.Dropout(config.hidden_dropout)\n        self.attention = GPTNeoXAttention(config)\n        self.mlp = GPTNeoXMLP(config)\n\n    def forward(\n        self,\n        hidden_states: Optional[torch.FloatTensor],\n        attention_mask: Optional[torch.FloatTensor] = None,\n        position_ids: Optional[torch.LongTensor] = None,\n        head_mask: Optional[torch.FloatTensor] = None,\n        use_cache: Optional[bool] = False,\n        layer_past: Optional[Tuple[torch.Tensor]] = None,\n        output_attentions: Optional[bool] = F

In [3]:
decomp.model[0].forward?

Signature:      
decomp.model[0].forward(
    input_ids: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.FloatTensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    head_mask: Optional[torch.FloatTensor] = None,
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
    labels: Optional[torch.LongTensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple, transformers.modeling_outputs.CausalLMOutputWithPast]
Call signature:  decomp.model[0].forward(*args, **kwargs)
Type:            partial
String form:    
functools.partial(<function add_hook_to_module.<locals>.new_forward at 0x72008815b760>, GPTNeoXFo <...> ntwise_affine=True)
           )
           (embed_out): Linear(in_features=512, out_features=50304, bias=False)
           ))

In [4]:
decomp.load(reload=False)

Loading dataset from cache
Got embeddings
Using saved correlation for layer 0
Using saved correlation for layer 1
Using saved correlation for layer 2
Using saved correlation for layer 3
Using saved correlation for layer 4
Using saved correlation for layer 5


## Lets just look at the highest cliques

In [5]:
decomp.internal_correlations[1]

array([[1.        , 0.        , 0.55996289, ..., 0.55218228, 0.55726815,
        0.54649686],
       [0.        , 1.        , 0.420132  , ..., 0.54260334, 0.55746129,
        0.43269434],
       [0.55996289, 0.420132  , 1.        , ..., 0.5145273 , 0.58364553,
        0.39925619],
       ...,
       [0.55218228, 0.54260334, 0.5145273 , ..., 1.        , 0.57276563,
        0.52797111],
       [0.55726815, 0.55746129, 0.58364553, ..., 0.57276563, 1.        ,
        0.        ],
       [0.54649686, 0.43269434, 0.39925619, ..., 0.52797111, 0.        ,
        1.        ]])

## Build Up a "Concept" Lattice Using Graph Restrictions

> TODO: this is for latter...

In [6]:
import src.graph as graph
import networkx as nx

LAYER = 3

G = graph.di_graph_from_correlations(decomp.internal_correlations[LAYER])
MAX_CLIQUE_SIZE = 1_000

In [8]:
GG = graph.sparsify_weighted_digraph(G, degree_upperbound=50)

In [12]:
for j in range(1_024):
    c = GG.get_edge_data(0, j)
    if c is not None:
        print(c)

{'weight': 0.8521209974147577}
{'weight': 0.8162661745812154}
{'weight': 0.7875908366274808}
{'weight': 0.9736834434621171}
{'weight': 0.7978421185982739}
{'weight': 0.9630868645920452}
{'weight': 0.7848217348702783}
{'weight': 0.8309435349882173}
{'weight': 0.8466209110905328}
{'weight': 0.7892141804916221}
{'weight': 0.8238873704295807}
{'weight': 0.8249793158188582}
{'weight': 0.9295728856730935}
{'weight': 0.7817297580903736}
{'weight': 0.8035910877524983}
{'weight': 0.8094992522304162}
{'weight': 0.780198343920196}
{'weight': 0.7894159773808109}
{'weight': 0.8018332126288977}
{'weight': 0.8466276376535058}
{'weight': 0.7833844925817222}
{'weight': 0.8762940225519235}
{'weight': 0.8060373144869987}
{'weight': 0.7847163520503685}
{'weight': 0.8033511736731294}
{'weight': 0.8379077698528901}
{'weight': 0.8436836452590063}
{'weight': 0.7810526174177621}
{'weight': 0.882417437044976}
{'weight': 0.7973084779357523}
{'weight': 0.7856042583627995}
{'weight': 0.8001695093869187}
{'weight':

In [ ]:
next(i)

[19, 962, 932, 292, 564]

In [ ]:
GG.get_edge_data

In [ ]:
import importlib
import src.utils as utils
importlib.reload(utils)

decomp.tune_clique((0.0, clique), 3)

<module 'src.utils' from '/home/lev/code/research/ai/lattice_cluster_paths/src/utils.py'>

In [ ]:
# import networkx as nx

# test_at = 100
# GG = graph.sparsify_weighted_graph(G, degree_upperbound=test_at)

# cliques = []

# for node in range(N_DIMS * 2):
#     if node % 50 == 0:
#         print("On node", node)
#     c_it = nx.find_cliques(GG, nodes=[node])
#     c_in_curr = []
#     for c in c_it:
#         c_in_curr.append(c)
#         if len(c_in_curr) > 50:
#             break
#     cliques += c_in_curr
#         # if len(cliques) > 10_000:
#         #     break

## Score a cell

In [ ]:
decomp.internal_correlations[0]

array([[1.        , 0.        , 0.50411391, ..., 0.41980654, 0.47819793,
        0.52677501],
       [0.        , 1.        , 0.51517917, ..., 0.56102004, 0.50169823,
        0.55460722],
       [0.50411391, 0.51517917, 1.        , ..., 0.47680901, 0.5166343 ,
        0.52601947],
       ...,
       [0.41980654, 0.56102004, 0.47680901, ..., 1.        , 0.40289529,
        0.57754201],
       [0.47819793, 0.50169823, 0.5166343 , ..., 0.40289529, 1.        ,
        0.        ],
       [0.52677501, 0.55460722, 0.52601947, ..., 0.57754201, 0.        ,
        1.        ]])

In [ ]:
import src.decorrelate as decc
import src.kernel as kernel
importlib.reload(decc)
importlib.reload(utils)
importlib.reload(kernel)

LAYER = 2
NEURON = 2

# TODO: we **need** to do something like curr-clique removal so we get more interesting cliques...
# Also the per-neuron thing makes no sense...

# TODO: change size of subset??
decomp.scores_for_neuron(LAYER, NEURON, degree_upperbound=400, n_features_per_neuron=3)

GOT CLIQUES WITH LENGTH [9, 9, 10] [(23.79445227225935, [2, 771, 128, 217, 389, 593, 657, 661, 785]), (25.24844098956523, [2, 771, 128, 217, 389, 593, 657, 687, 936]), (29.81072255571707, [2, 771, 128, 217, 389, 593, 657, 687, 785, 613])]
Got tuned clique [0.0, 0.8, 0.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.6]
Got tuned clique [0.0, 0.0, 0.0, 0.4, 1.0, 1.0, 1.0, 0.6, 0.0]
Got tuned clique [0.0, 0.8, 0.0, 0.6, 1.0, 1.0, 1.0, 1.0, 0.6, 1.0]
Finished for neuron 2 2
